# Showcase of Bybit Future - Future calc
The Bybitmaster has custom methods that I coded, but can also make normal request to the Bybit API with the session attribute.

In [ ]:
# Inside of the module, we use absolute imports
# Because this context is different, we need to add inside the PYTHONPATH the path of the module
import sys
sys.path.append('Bybit')

from Bybit.greekMaster import SpotFutStrategos
import Bybit.utils
import Bybit.simulator
Bybit.utils.ColorFormatter.configure_logging(run_name="save_klines.log", verbose=1)

# Create a Bybit master
master = SpotFutStrategos(demo=True)
sim = Bybit.simulator.Simulator()
master.fetcher.get_USDC_BTC()

In [ ]:
# Get all futures BTC gaps
DataWrangler = master.fetcher.all_gaps_pd(coin="BTC",inverse=True, spot=True, perpetual=True, applyFees=True)

# Format the DataFrame
pretty = True
if pretty:
    DataWrangler["Gap"] = DataWrangler["Gap"].map(lambda x: f"$ {x:.2f}")
    DataWrangler["Coeff"] = DataWrangler["Coeff"].map(lambda x: f"{x:.2f} %")
    DataWrangler["APR"] = DataWrangler["APR"].map(lambda x: f"{x:.2f} %")
    DataWrangler["CumVolume"] = DataWrangler["CumVolume"].map(Bybit.utils.format_volume)

DataWrangler

In [ ]:
await master.client.fetcher.get_history_pd("BTC-07FEB25", category="linear", interval="15", dest="store")
await master.client.fetcher.get_history_pd("BTC-07FEB25", category="linear", interval="5", dest="store")
await master.client.fetcher.get_history_pd("BTC-07FEB25", category="linear", interval="1", dest="store")

# master.client.fetcher.session.get_instruments_info(symbol="BTCUSDH25", category="inverse")

# Bybit.utils.get_date(1743148800000)
# start = Bybit.utils.get_date(1726214400000)

## Tester
Just to run some tests

In [ ]:
# 1 - Simple enter position logic (given a future and a perpetual)
# 2 - Simple exit position logic (given a future and a perpetual). Can also exit when reaching a satisfying unexpected gap
# 3 - Logic to activate both operations (given a future and a perpetual). Enter again when reaching a satisfying unexpected gap
# 4 - Logic to find the best future and perpetual to trade
# 5 - Logic to keep the loop running
# 6 - Monitoring features
# 7 - Use systemctl to launch 24/7
# We use sockets, once we selected the best local future, compared to their average gap.
# Shared state to store the latest updates
import datetime
print(datetime.datetime.now().microsecond)
(1734816631000 - datetime.datetime.now().microsecond) / 1000

## Fetching time :)

Getting 15, 5 and 1 minute candles for everything since 01/01/2024

In [ ]:
# Future contracts
await master.client.fetcher.save_klines(dest="store")

## Plot testing

Part to test the plot of the data.


In [ ]:
# Yes
# master.client.fetcher.get_futureNames(quoteCoins=["USDT"])

## Long Spot x Short Future

Spot USDC is 0% fees and Future is 0.0550% taker fees (0% with contract delivery).
Problem is that spot cannot be used with leverage. Could borrow but that would be awkward. (tried it)

In [ ]:
Bybit.utils.load_klines_parquet("store/BTC-20DEC24_1.parquet", pretty=True)

sim.plot_compare("store/BTCUSDT_1.parquet", "store/BTC-14FEB25_1.parquet")

In [ ]:
# Long position, then short position
sim.plot_compare("store/BTCUSDT_1.parquet", "store/BTC-15NOV24_1.parquet")

## Long Perpetual x Short Future

Perpetual USDT is 0.0550% taker fees and Future is 0.0550% taker fees (0% with contract delivery).
Yes we have fees, but wen can use leverage on both sides.

We use Perpetual USDT because there is more liquidity.

In [ ]:
sim.plot_compare("store/BTCPERP_5.parquet", "store/BTC-29NOV24_5.parquet")

In [ ]:
sim.plot_compare("store/BTCUSDT_5.parquet", "store/BTC-29NOV24_5.parquet")

## Long Spot x Short Inverse Future

What if I buy spot and collateralize it to short inverse future?
We effectively capture the difference.

In [ ]:
# Keep going when get_history_pd file names are changed

## OKX 20x Leverage test

I launched a test with OKX with a short on BTCUSDT-27JUN25 and long on BTCUSDT-29NOV24.

Following section will track the the two contracts' data

In [ ]:
sim.plot_compare("store/BTC-29NOV24_15.parquet", "store/BTC-27JUN25_15.parquet", dateLimit="2024-05-06")